In [39]:
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from keras.optimizers import SGD
from keras.constraints import maxnorm
from keras.layers import Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [4]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [5]:
#load dataset
from sklearn.datasets import load_boston
boston,price = load_boston(return_X_y=True)

Y=price.reshape(506,)
X=boston.astype(float)
print (X.shape)
print (Y.shape)

(506, 13)
(506,)


In [6]:
model = Sequential()
model.add(Dense(30, input_dim=13, init='normal' ,activation='relu'))
model.add(Dense(15, init='normal', activation='relu'))
model.add(Dense(1, init='normal'))
# Compile model
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae'])

In [7]:
# Fit the model
model.fit(X, Y, nb_epoch=300, batch_size=10)

Epoch 1/300
506/506 [==============================] - 0s - loss: 478.9033 - mean_absolute_error: 19.4927     
Epoch 2/300
506/506 [==============================] - 0s - loss: 139.8446 - mean_absolute_error: 8.9755     
Epoch 3/300
506/506 [==============================] - 0s - loss: 104.9336 - mean_absolute_error: 7.5282     
Epoch 4/300
506/506 [==============================] - 0s - loss: 87.2788 - mean_absolute_error: 6.7434     
Epoch 5/300
506/506 [==============================] - 0s - loss: 74.0565 - mean_absolute_error: 6.1394     
Epoch 6/300
506/506 [==============================] - 0s - loss: 69.7227 - mean_absolute_error: 6.0138     
Epoch 7/300
506/506 [==============================] - 0s - loss: 66.0468 - mean_absolute_error: 5.7924     
Epoch 8/300
506/506 [==============================] - 0s - loss: 61.5399 - mean_absolute_error: 5.5432     
Epoch 9/300
506/506 [==============================] - 0s - loss: 59.0228 - mean_absolute_error: 5.4458     
Epoch 10/300
50

In [8]:
#evaluate model
score=model.evaluate(X,Y,batch_size=10,verbose=1,sample_weight=None)
print(" Mean Absolute Error: %.2f " % (score[1]))

460/506 [==========================>...] - ETA: 0s Mean Absolute Error: 2.68 


In [27]:
#Question 2 - Grid Search for Optimizer Functions
def createModel(optimizer='adam', learn_rate=0.01, momentum=0, activation='relu',dropout_rate=0.0, weight_constraint=0):
    model = Sequential()
    model.add(Dense(30, input_dim=13, init='normal' ,activation='relu'))
    model.add(Dense(15, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae'])
    return model

In [28]:
#Downsampling the data
X_T=X[1:100,:]
Y_T=Y[1:100,]
print (X_T.shape)
print (Y_T.shape)

(99, 13)
(99,)


In [29]:
# evaluate model
estimator = KerasRegressor(build_fn=createModel, nb_epoch=5, batch_size=10,verbose=0)
#kfold = KFold(n_splits=5, random_state=seed)
#results = cross_val_score(estimator, X_T, Y_T, cv=kfold)

#print(results)
#print("Results: %.2f (%.2f)" % (results.mean(), results.std()))


In [30]:
# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=estimator, param_grid=param_grid, n_jobs=1,scoring='neg_mean_absolute_error')
grid_result = grid.fit(X_T, Y_T)

In [31]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -8.197482 using {'optimizer': 'RMSprop'}
-12.320509 (2.327505) with: {'optimizer': 'SGD'}
-8.197482 (2.333108) with: {'optimizer': 'RMSprop'}
-12.712444 (2.883628) with: {'optimizer': 'Adagrad'}
-13.180811 (5.706599) with: {'optimizer': 'Adadelta'}
-10.621854 (4.115492) with: {'optimizer': 'Adam'}
-12.066472 (4.864682) with: {'optimizer': 'Adamax'}
-10.179547 (0.256207) with: {'optimizer': 'Nadam'}


In [32]:
# define the grid search parameters
learn_rate = [0.1, 0.2, 0.3]
momentum = [0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=estimator, param_grid=param_grid, n_jobs=1,scoring='neg_mean_absolute_error')
grid_result = grid.fit(X_T, Y_T)

In [33]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -7.092687 using {'momentum': 0.9, 'learn_rate': 0.1}
-15.618842 (3.863621) with: {'momentum': 0.6, 'learn_rate': 0.1}
-7.193136 (1.605399) with: {'momentum': 0.8, 'learn_rate': 0.1}
-7.092687 (3.209418) with: {'momentum': 0.9, 'learn_rate': 0.1}
-9.859799 (2.109321) with: {'momentum': 0.6, 'learn_rate': 0.2}
-9.905955 (4.056334) with: {'momentum': 0.8, 'learn_rate': 0.2}
-12.883302 (5.787058) with: {'momentum': 0.9, 'learn_rate': 0.2}
-8.984422 (1.669774) with: {'momentum': 0.6, 'learn_rate': 0.3}
-10.794084 (4.178972) with: {'momentum': 0.8, 'learn_rate': 0.3}
-13.768197 (4.651002) with: {'momentum': 0.9, 'learn_rate': 0.3}


In [35]:
# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=estimator, param_grid=param_grid, n_jobs=1, scoring='neg_mean_absolute_error')
grid_result = grid.fit(X_T, Y_T)

In [36]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -9.873848 using {'activation': 'hard_sigmoid'}
-13.145561 (6.581714) with: {'activation': 'softmax'}
-10.305245 (2.635532) with: {'activation': 'softplus'}
-12.234603 (5.425744) with: {'activation': 'softsign'}
-10.034799 (3.644700) with: {'activation': 'relu'}
-11.358992 (4.708074) with: {'activation': 'tanh'}
-13.903073 (2.252260) with: {'activation': 'sigmoid'}
-9.873848 (4.250643) with: {'activation': 'hard_sigmoid'}
-10.906476 (4.238910) with: {'activation': 'linear'}


In [38]:
# define the grid search parameters
weight_constraint = [1, 2,3]
dropout_rate = [0.0, 0.1, 0.3, 0.5, 0.8]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)
grid = GridSearchCV(estimator=estimator, param_grid=param_grid, n_jobs=1, scoring='neg_mean_absolute_error')
grid_result = grid.fit(X_T, Y_T)# summarize results

KeyboardInterrupt: 

In [40]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -9.873848 using {'activation': 'hard_sigmoid'}
-13.145561 (6.581714) with: {'activation': 'softmax'}
-10.305245 (2.635532) with: {'activation': 'softplus'}
-12.234603 (5.425744) with: {'activation': 'softsign'}
-10.034799 (3.644700) with: {'activation': 'relu'}
-11.358992 (4.708074) with: {'activation': 'tanh'}
-13.903073 (2.252260) with: {'activation': 'sigmoid'}
-9.873848 (4.250643) with: {'activation': 'hard_sigmoid'}
-10.906476 (4.238910) with: {'activation': 'linear'}


In [41]:
# KerasRegressor wrapper
def create_model_3():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='hard_sigmoid'))
    #model.add(Dropout(dropout_rate))
    model.add(Dense(1, kernel_initializer='normal', activation ='hard_sigmoid'))
    optimizer = SGD(lr=0.1, momentum=0.9)
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='RMSprop')
    return model

In [44]:
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=create_model_3, nb_epoch=100, batch_size=5, verbose=0)
pipeline = Pipeline(estimator)
kfold = KFold(n_splits=10, random_state=seed)
#results = cross_val_score(estimator, X_T, Y_T, cv=kfold)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

TypeError: 'KerasRegressor' object is not iterable

In [ ]:

# evaluate model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=create_model_3, epochs=5, batch_size=10, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X_T, Y_T, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [ ]:
# smaller hidden layer
def create_small_model():
# create model
    model = Sequential()
    model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='hard_sigmoid'))
    model.add(Dense(1, kernel_initializer='normal', activation='hard_sigmoid'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='RMSprop')
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=create_small_model, epochs=5, batch_size=10, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X_T, Y_T, cv=kfold)
print("Smaller: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [ ]:
# deeper model
def create_large_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='hard_sigmoid'))
    model.add(Dense(7, kernel_initializer='normal', activation='hard_sigmoid'))
    model.add(Dense(1, kernel_initializer='normal'))
    #model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='RMSprop')
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=create_large_model, epochs=5, batch_size=10, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=5, random_state=seed)
results = cross_val_score(pipeline, X_T, Y_T, cv=kfold)
print("Deeper: %.2f (%.2f) MSE" % (results.mean(), results.std()))


In [ ]:
# wider hidden layer
def create_wider_model():
# create model
    model = Sequential()
    model.add(Dense(20, input_dim=13, kernel_initializer='normal', activation='hard_sigmoid'))
    model.add(Dense(1, kernel_initializer='normal', activation='hard_sigmoid'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='RMSprop')
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=create_wider_model, epochs=5, batch_size=10, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
#results = cross_val_score(pipeline, X, Y, cv=kfold)
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X_T, Y_T, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))